# Open Weather Map
Get a look on the data of openweathermap.org.
## Prerequisits

In [61]:
import requests
import user_data as ud
import pandas as pd

def del_lst(x):
    return x[0]

url = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lng}' \
      '&units=metric&APPID={key}'.format(lat=ud.lat, lng=ud.lng, key=ud.owm)

## API
Get data from API and safe json file.

In [62]:
response = requests.get(url)
data = response.json()
data.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'minutely', 'hourly', 'daily'])

## Current Weather
- Timezone is UTC
- temp, feels_like, dew_point: °C
- pressure: hPa
- humidity, clouds: %
- uvi: UV Index
- visibility: m
- wind_speed: m / s
- wind_deg: °

In [63]:
keys = ['timezone', 'timezone_offset', 'current']
time = ['dt', 'sunrise', 'sunset']

cur = pd.DataFrame(list(map(data.get, keys)), keys).T
cur = cur.join(pd.DataFrame(cur.current[0]))
cur = cur.join(pd.DataFrame(cur.weather[0], [0]))
cur = cur.drop(columns=['current', 'weather'])

cur['timezone_offset'] = pd.to_timedelta(cur.timezone_offset, 's')
cur[time] = cur[time].apply(pd.to_datetime, unit='s')
cur = cur.set_index('dt')
cur

,timezone,timezone_offset,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,
2021-03-09 20:49:08,Europe/Berlin,0 days 01:00:00,2021-03-09 05:58:19,2021-03-09 17:21:29,1.97,-1.31,1018,93,0.96,0,75,5000,2.06,90,701,Mist,mist,50n


### Alerts

In [64]:
# ale = pd.DataFrame(data['alerts'], ['german', 'english'])
#
# time = ['start', 'end']
#
# ale[time] = ale[time].apply(pd.to_datetime, unit='s')
# ale

## Minutely Data
- 1 hour
- 60 observations
- precipitation: mm

In [65]:
mnt = pd.DataFrame(data['minutely'])
mnt = mnt.set_index('dt')
mnt.index = pd.to_datetime(mnt.index, unit='s')
mnt

,precipitation
dt,
2021-03-09 20:50:00,0
2021-03-09 20:51:00,0
2021-03-09 20:52:00,0
2021-03-09 20:53:00,0
2021-03-09 20:54:00,0
...,...
2021-03-09 21:46:00,0
2021-03-09 21:47:00,0
2021-03-09 21:48:00,0


## Hourly Data
- 2 days
- 48 observations
- pop: %

In [66]:
hr = pd.DataFrame(data['hourly'])
hr = hr.set_index('dt')
hr.index = pd.to_datetime(hr.index, unit='s')
hr = hr.join(pd.DataFrame(hr.weather.apply(del_lst).tolist(), hr.index))
hr = hr.drop(columns='weather')
hr

,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,pop,rain,id,main,description,icon
dt,,,,,,,,,,,,,,,,
2021-03-09 20:00:00,1.97,-1.38,1018,93,0.96,0.00,75,10000,2.15,83,0.00,NaN,803,Clouds,broken clouds,04n
2021-03-09 21:00:00,2.15,-1.01,1018,91,0.84,0.00,87,10000,1.86,97,0.00,NaN,804,Clouds,overcast clouds,04n
2021-03-09 22:00:00,1.99,-1.29,1019,90,0.53,0.00,94,10000,1.96,117,0.00,NaN,804,Clouds,overcast clouds,04n
2021-03-09 23:00:00,1.25,-1.86,1019,92,0.10,0.00,93,10000,1.63,148,0.00,NaN,804,Clouds,overcast clouds,04n
2021-03-10 00:00:00,0.68,-2.45,1018,94,-0.15,0.00,92,10000,1.60,149,0.00,NaN,804,Clouds,overcast clouds,04n
2021-03-10 01:00:00,0.32,-3.14,1018,96,-0.50,0.00,50,10000,2.06,139,0.00,NaN,802,Clouds,scattered clouds,03n
2021-03-10 02:00:00,-0.05,-3.90,1018,98,-0.74,0.00,39,10000,2.60,136,0.00,NaN,802,Clouds,scattered clouds,03n
2021-03-10 03:00:00,-0.54,-4.38,1018,98,-1.14,0.00,27,10000,2.48,162,0.00,NaN,802,Clouds,scattered clouds,03n
2021-03-10 04:00:00,-0.54,-4.55,1017,98,-1.13,0.00,45,10000,2.73,163,0.00,NaN,802,Clouds,scattered clouds,03n


## Daily Data
- 1 week
- 8 observations

In [67]:
time = ['dt', 'sunrise', 'sunset']
dly = pd.DataFrame(data['daily'])
dly[time] = dly[time].apply(pd.to_datetime, unit='s')

dly = dly.join(pd.DataFrame(dly.temp.to_list()))
dly = dly.join(pd.DataFrame(dly.feels_like.to_list()).add_suffix('_feels_like'))
dly = dly.join(pd.DataFrame(dly.weather.apply(del_lst).to_list()))

dly = dly.drop(columns=['temp', 'feels_like', 'weather'])
dly = dly.set_index('dt')
dly

,sunrise,sunset,pressure,humidity,dew_point,wind_speed,wind_deg,clouds,pop,rain,...,eve,morn,day_feels_like,night_feels_like,eve_feels_like,morn_feels_like,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,,,
2021-03-09 11:00:00,2021-03-09 05:58:19,2021-03-09 17:21:29,1017,91,2.53,3.15,70,100,1.00,6.06,...,3.26,2.96,-0.05,-1.29,-0.27,-1.17,501,Rain,moderate rain,10d
2021-03-10 11:00:00,2021-03-10 05:55:55,2021-03-10 17:23:21,1016,87,3.03,6.14,182,100,1.00,2.31,...,6.05,-0.47,-0.91,-2.48,0.45,-4.73,500,Rain,light rain,10d
2021-03-11 11:00:00,2021-03-11 05:53:30,2021-03-11 17:25:13,992,67,4.06,14.22,238,96,1.00,9.41,...,6.64,8.49,-1.53,-2.00,-0.74,0.14,501,Rain,moderate rain,10d
2021-03-12 11:00:00,2021-03-12 05:51:05,2021-03-12 17:27:05,1003,72,2.14,9.87,238,58,1.00,6.03,...,5.68,3.43,-1.85,-1.89,-2.26,-4.75,501,Rain,moderate rain,10d
2021-03-13 11:00:00,2021-03-13 05:48:40,2021-03-13 17:28:56,994,90,4.98,10.55,221,100,1.00,9.00,...,5.80,3.91,-2.13,-0.51,-0.55,-3.31,501,Rain,moderate rain,10d
2021-03-14 11:00:00,2021-03-14 05:46:14,2021-03-14 17:30:47,1001,72,2.55,9.06,289,95,0.97,1.52,...,3.49,2.88,-0.73,-1.98,-1.73,-3.24,500,Rain,light rain,10d
2021-03-15 11:00:00,2021-03-15 05:43:48,2021-03-15 17:32:38,1014,68,0.93,9.05,316,57,0.31,0.78,...,4.20,3.50,-1.81,-2.99,-2.39,-1.65,500,Rain,light rain,10d
2021-03-16 11:00:00,2021-03-16 05:41:22,2021-03-16 17:34:29,1024,71,1.12,4.71,289,97,0.56,0.71,...,3.91,3.63,0.68,0.08,1.07,-2.37,500,Rain,light rain,10d
